In [ ]:
import numpy as np
# from tqdm.notebook import tqdm
from tqdm import tqdm
import matplotlib.pyplot as plt
import librosa
from pprint import pprint
import scipy.ndimage
from IPython.display import display, Audio
from estimator import ActivationLearner
import scipy.signal
import logging
from unmixdb import UnmixDB
from beat_track import beat_stft

logging.basicConfig(level=logging.INFO)
logging.getLogger("activation_learner").setLevel(logging.DEBUG)

In [ ]:
# Load audios

FS = 22050
# input_paths = ["linear-mix-1.wav", "linear-mix-2.wav", "linear-mix.wav"]
# input_paths = ["original.wav", "original.wav"]
# input_paths = ["original.wav", "boucled.wav"]
# input_paths = ["amen.wav", "high.wav", "nuttah.wav"]
# input_paths = ["nuttah.wav", "nuttah-timestretch.wav"]
# inputs = [librosa.load(i, sr=FS)[0] for i in inputs]


## load unmixdb
unmixdb = UnmixDB("/data2/anasynth_nonbp/schwarz/abc-dj/data/unmixdb-zenodo")
print(unmixdb.timestretches)
print(unmixdb.fxes)
mixes = dict(filter(lambda i: i[1].timestretch == "stretch" and i[1].fx=="none", unmixdb.mixes.items()))
mix = list(mixes.values())[5]
pprint(mix)
inputs = [unmixdb.refsongs[track['name']].audio(sr=FS) for track in mix.tracks]
inputs.append(mix.audio(sr=FS))

for i, mix in enumerate(inputs):
    print(i)
    display(Audio(mix, rate=FS))

In [ ]:
model = ActivationLearner(
    inputs,
    fs=FS,
    additional_dim=0,
    win_size = 1.0,
    hop_size = 0.25,
    n_mels = 512,
    col_mag_threshold=1e-8,
)
model.nmf.H[:200,200:] = 0

losses = []
for i in (pbar := tqdm(range(150))):
    loss = model.iterate()
    losses.append(loss)
    pbar.set_description(f"loss={loss:.2e}")
plt.figure(figsize=(8, 2))
plt.plot(losses)
plt.xlabel("iter")
plt.title("distance")
plt.yscale("log")
plt.show()
model.plot()

In [ ]:
plt.figure(figsize=(20,6))
for i, v in enumerate(model.volume()):
    plt.plot(v, label=f"track {i}")
plt.legend()

import soundfile as sf


def estimate_tracks():
    ret = []
    for i in range(len(refs) + 1):
        xi = model.reconstruct(i)
        ret.append(xi)
        print("track", i)
        display(Audio(xi, rate=FS))
        sf.write(f"estimated-{i}.wav", xi / xi.max(), FS)
    return ret